<a href="https://colab.research.google.com/github/KevinWahle/ASSD-TPF/blob/master/ASSD_Dialization%2BTranscriber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dializator + Transcriber

El objetivo de este notebook es: ...


In [20]:
debugging=1

## Descarga e importación de librerías


In [21]:
import os
os.system("pip install git+https://github.com/openai/whisper.git")
os.system("pip install git+https://github.com/resemble-ai/Resemblyzer.git")
os.system("pip install pathlib")
os.system("pip install librosa==0.9.2")

import whisper
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
from scipy.io import wavfile
import numpy as np

## Dialización

### Carga de Oradores

In [86]:
wav_fpath = "/content/dibu.wav"
wav = preprocess_wav(wav_fpath)
sampling_rate=16000

segments = [[1, 4], [6, 9]]                  
speaker_names = ["Interviewer", "Martinez"]
speaker_wavs = [wav[int(s[0] * sampling_rate):int(s[1] * sampling_rate)] for s in segments]

### Aplicación de la Red Neuronal

In [87]:
## Compare speaker embeds to the continuous embedding of the interview
# Derive a continuous embedding of the interview. We put a rate of 16, meaning that an 
# embedding is generated every 0.0625 seconds. It is good to have a higher rate for speaker 
# diarization, but it is not so useful for when you only need a summary embedding of the 
# entire utterance. A rate of 2 would have been enough, but 16 is nice for the sake of the 
# demonstration. 
# We'll exceptionally force to run this on CPU, because it uses a lot of RAM and most GPUs 
# won't have enough. There's a speed drawback, but it remains reasonable.
encoder = VoiceEncoder("cpu")
print("Running the continuous embedding on cpu, this might take a while...")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16) 

# Get the continuous similarity for every speaker. It amounts to a dot product between the 
# embedding of the speaker and the continuous embedding of the interview
speaker_embeds = [encoder.embed_utterance(speaker_wav) for speaker_wav in speaker_wavs]
similarity_dict = {name: cont_embeds @ speaker_embed for name, speaker_embed in 
                   zip(speaker_names, speaker_embeds)}
                  
if debugging:
  print(similarity_dict)

Loaded the voice encoder model on cpu in 0.01 seconds.
Running the continuous embedding on cpu, this might take a while...
{'Interviewer': array([0.83435327, 0.83405817, 0.80782056, 0.81869924, 0.8048623 ,
       0.8040961 , 0.81363964, 0.8236759 , 0.8124765 , 0.7894388 ,
       0.78801465, 0.80411685, 0.822531  , 0.8303324 , 0.8579163 ,
       0.8687671 , 0.8935776 , 0.89803374, 0.9001    , 0.9075572 ,
       0.9248593 , 0.9293411 , 0.9374424 , 0.9391928 , 0.9395311 ,
       0.9382777 , 0.944386  , 0.94994783, 0.95681   , 0.95442855,
       0.95617497, 0.95142865, 0.9558789 , 0.9492326 , 0.930969  ,
       0.94376785, 0.9354578 , 0.9144671 , 0.91491866, 0.90962076,
       0.91571015, 0.91186655, 0.9037841 , 0.901852  , 0.8909137 ,
       0.8816436 , 0.8613314 , 0.8550217 , 0.8592711 , 0.8305787 ,
       0.81934834, 0.8191619 , 0.8147546 , 0.79714584, 0.79845834,
       0.7870809 , 0.77327275, 0.770459  , 0.7807823 , 0.77247304,
       0.76645184, 0.74410325, 0.7445687 , 0.72530586, 0.

### Prefiltrado de la dialización
En esta etapa se busca: ... 


In [94]:
talk_names =[]
MIN_PROB = 50/100


for i in range(len(similarity_dict[speaker_names[0]])):     # Create an array with speaker predominance in each sample
    prev = 0
    for names in speaker_names:
        prev= max(similarity_dict[names][i], prev)
        if(prev==similarity_dict[names][i]): name = names
    if prev<MIN_PROB: name = None
    talk_names.append(name)

# Si tuve 1 valor de 5 diferente, corregilo
for i in range(len(talk_names)):                            # Soft change matrix 
    if 1<i<len(talk_names)-3 and talk_names[i-1]==talk_names[i-2]==talk_names[i+1]==talk_names[i+2]:
        talk_names[i]=talk_names[i-1]

# Crea arreglo con tiempos y nombres [[start,end,speaker]]
name=talk_names[0]
speakers_time = []
prev_name=0
io=0

for i in range(len(talk_names)):
    name=talk_names[i]
    if name!=prev_name:
        if prev_name!=None:
          if i<len(talk_names):
            speakers_time.append([io+1,i+1,prev_name,""])
          else:
            speakers_time.append([io+1,i,prev_name,""])            
        io=i+1
    prev_name=name

# Si el delta_tiempo es menor a MIN_TIME, sacamelo
# Si son tempos cercanos (menor a MIN_TIME), hace merge
MERGE_TIME = 5
DEL_TIME = 5

time_processed=[]
for i in range(len(speakers_time)):
    #Do merge
    if len(time_processed) and speakers_time[i][2]==time_processed[-1:][0][2] and (speakers_time[i][0]-time_processed[-1:][0][1])<=MERGE_TIME:
        temp = time_processed[-1:][0][0]            # Guardo de la iteración anterior 
        time_processed.pop()                        # Saco el tiempo viejo
        time_processed.append([temp, speakers_time[i][1], speakers_time[i][2], ""]) # Agrego el merge
    
    #Delete time
    elif (speakers_time[i][1]-speakers_time[i][0])>MIN_TIME:
        if len(time_processed) and time_processed[-1:][0][1]-time_processed[-1:][0][0]<=DEL_TIME:
            time_processed.pop()                    # Si el de la iteración anteior era menor, lo saco
        time_processed.append(speakers_time[i])     # Agrego el tiempo si es mayor a mintime
    
    #Append and check in next iteration
    elif i<len(speakers_time)-1:
        if len(time_processed) and time_processed[-1:][0][1]-time_processed[-1:][0][0]<=DEL_TIME:
            time_processed.pop()                    # Si el de la iteración anteior era menor, lo saco
        time_processed.append(speakers_time[i])

if debugging:
  print(time_processed)

[[2, 77, 'Interviewer', ''], [78, 553, 'Martinez', ''], [554, 644, 'Interviewer', ''], [652, 772, 'Martinez', ''], [773, 830, 'Interviewer', ''], [833, 915, 'Martinez', '']]


### Visualizacion

In [89]:
## Run the interactive demo
#interactive_diarization(similarity_dict, wav, wav_splits)

## Speech to Text

### Creación de wavs por orador

In [95]:
# Creacion de los wavs ya separados en los distintos tramos por orador
def wavCreation(slices):
  magic_number=960    
  start_offset=8
  end_offset=10

  if not os.path.exists("/content/temp"):
    os.makedirs("/content/temp")

  for slice_index in range(len(slices)):
    sliceStart=(slices[slice_index][0]+start_offset)*magic_number
    sliceEnd=(slices[slice_index][1]+end_offset)*magic_number
    wavfile.write("/content/temp/slice"+str(slice_index)+".wav", sampling_rate, wav[sliceStart:sliceEnd])

wavCreation(time_processed)

### Transcripción

In [96]:
def speech2Text(slices):
    model = whisper.load_model("base")          #TODO: Revisar tamaño modelo
    for slice_index in range(len(slices)):
        result = model.transcribe("/content/temp/slice"+str(slice_index)+".wav")
        slices[slice_index][3]=result['text']
        
        if debugging:
          print(slices[slice_index][2]+":")
          print(result['text'])
          #input("press enter to continue:")
        
        os.remove("/content/temp/slice"+str(slice_index)+".wav")


speech2Text(time_processed)

Interviewer:
 the hero of all Argentinean. Can you sum up your feelings after that tonight?
Martinez:
 It was a tricky game and we were up for control the game really well. We went to Nielab, basically we controlled the game, the referee was just giving everything for them. All of a sudden they get a good header, a good flick, which I couldn't see here. And then it turned upside down, the referee was giving everything for them, he gave 10 minutes. For no reason, 10 minutes, it was giving three kick outside the box for them like two, three times. He just wanted them to score that, that's basically here. So hopefully we don't have that referee anymore, he's useless.
Interviewer:
 Okay, but when you got to the penalty shootout, what's the thinking going into the spot kicks to have it all?
Martinez:
 I have a friend, Van Gaal saying we're going to advantage impenetives. If we go to impenetives we win. I think he needs to keep his mouth shut.
Interviewer:
 Okay, going forward now Croatia an

## Resultado final


In [97]:
def showConversation(data):
  for i in range(len(data)):
    print(data[i][2])
    print(data[i][3])

showConversation(time_processed)

Interviewer
 the hero of all Argentinean. Can you sum up your feelings after that tonight?
Martinez
 It was a tricky game and we were up for control the game really well. We went to Nielab, basically we controlled the game, the referee was just giving everything for them. All of a sudden they get a good header, a good flick, which I couldn't see here. And then it turned upside down, the referee was giving everything for them, he gave 10 minutes. For no reason, 10 minutes, it was giving three kick outside the box for them like two, three times. He just wanted them to score that, that's basically here. So hopefully we don't have that referee anymore, he's useless.
Interviewer
 Okay, but when you got to the penalty shootout, what's the thinking going into the spot kicks to have it all?
Martinez
 I have a friend, Van Gaal saying we're going to advantage impenetives. If we go to impenetives we win. I think he needs to keep his mouth shut.
Interviewer
 Okay, going forward now Croatia and sem

#TODO
* Mejorar mergeado de cortes y revisar la minima cant de muestras por corte
* Integrar UI